# Introducción a Conexión y Carga de Datos desde API Python a Plataforma GIS
## Paso 0: Instalar el módulo de python en el entorno conda

[Cómo instalar usando Anaconda](https://developers.arcgis.com/python/guide/install-and-set-up/#Installation-using-Anaconda-for-Python-Distribution)


## Paso 1: Importar el módulo de arcgis y conectarse al portal de la organización

[Documentación](https://developers.arcgis.com/python/guide/using-the-api/)

In [ ]:
import csv
import pandas as pd
import arcgis
from arcgis.gis import GIS
from arcgis.geometry import project
from IPython.display import display

gis = GIS("https://gis.ufro.cl/portal", 'initialadmin', 'Ufroportal.2021', verify_cert=False)

#Modo de prueba: Obtener un servicio de datos de contenido
demodata=gis.content.get('62ea18749a9649e1ad0b8551e75953a5')

## Paso 2: Establecer una consulta de datos
En este proceso se pueden utilizar, opcionalmente, elementos de librería pandas para tabular datos
1. Obtener la capa de interes
2. Realizar una query de datos
Actualmente la capa de prueba tiene tres elementos: Observaciones (0) Rutas (1) y Zonas (2). En este ejercicio se obtendrá la capa de observaciones y se le agregará un punto

In [ ]:
#Lista las capas del servicio, retorna un arreglo
capas=demodata.layers
#Se selecciona la capa que corresponde según el arreglo
puntos=capas[0]

#Operación query, realiza una consulta de datos al servicio y puede ser usada como una tabla 
consulta=puntos.query(out_fields="id,descripcion,tipo")
consulta.sdf

## Paso 3: Formular una acción de edición
Para editar, lo primero consiste en formar un diccionario sobre la base de la información existente de la consulta.
La tabla de observaciones tiene 3 campos:
1. id
2. descripcion
3. tipo

Pero ADEMAS es una capa de geometría de puntos por lo que requiere un X (longitud o coordenada este) e Y (latitud o coordenada norte)
El diccionario para ingresar un registro a la base de datos se compone de dos grupos "geometry" (con dos elementos x e y) y "attributes" (atributos alfanumericos)

In [ ]:
#Datos individuales

y=6500000
x=300000
id_dest="T003"
tipo=0
desc="Mi descripción"

#Reproyección de datos 
#32718: UTM WGS84 H18
#32719: UTM WGS84 H19
#4326: WGS84 Geográficas
#102100: WGS84 WebMercator (Servicios Web Estandar)
insr=32719
outsr=102100

#El comando proyect reproyecta una o múltiples geometrías de un sistema de coordenadas a otro, devuelve un arreglo de diccionarios [{}]
#Para este caso solo se reproyectará una coordenada por lo que la variable datosrp[0] representará al diccionario que tiene la geometría necesaria
datosrp=project(geometries=[{"x":x,"y":y}],in_sr=32719,out_sr=102100)

#Construcción de un diccionario para añadir un registro (1 diccionario por registro)
nf_dict= {
    "attributes":{
        'id':id_dest,
        'descripcion':desc,
        'tipo':tipo
        },
    "geometry":datosrp[0]}

#Operación de edición, esta se hace sobre la capa
puntos.edit_features(adds=[nf_dict])

#consulta actualizada
newconsulta=puntos.query(out_fields="id,descripcion,tipo")
newconsulta.sdf

## Paso 4: Cargar datos desde CSV
Con un CSV externo y python, se puede automatizar la lectura de datos desde archivos planos

In [ ]:
archivo='C:/Users/lzamudio/Documents/GitHub/UFROInducciones/input/data.csv'
csvfile=open(archivo,newline='',encoding='utf-8')
reader = csv.reader(csvfile)
#next(reader)
for row in reader:
    print(row)
csvfile.close()

## Paso 5: Combinar pasos 3 y 4

In [ ]:
archivo='C:/Users/lzamudio/Documents/GitHub/UFROInducciones/input/data.csv'
csvfile=open(archivo,newline='',encoding='utf-8')
reader = csv.reader(csvfile)
next(reader)

insr=4326
outsr=102100


for row in reader:
    y=row[0]
    x=row[1]
    id_dest=row[2]
    tipo=row[3]
    desc=row[4]
    datosrp=project(geometries=[{"x":x,"y":y}],in_sr=insr,out_sr=outsr)
    nf_dict= {
    "attributes":{
        'id':id_dest,
        'descripcion':desc,
        'tipo':tipo
        },
    "geometry":datosrp[0]}
    puntos.edit_features(adds=[nf_dict])

csvfile.close()

In [ ]:
newconsulta=puntos.query(out_fields="id,descripcion,tipo")
newconsulta.sdf